<a href="https://colab.research.google.com/github/Mutasar/Klasifikasi-Gambar/blob/main/template_submission_akhir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Klasifikasi Gambar: [Fruit Classification Dataset]
**Nama:** [Mutasar]

**Email:** [mutasarstmik@gmail.com]

**ID Dicoding:** [l319ywd083]

## Import Semua Packages/Library yang Digunakan

In [143]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [269]:
# Library yang sering digunakan
import os, shutil
import zipfile
import random
from random import sample
import shutil
from shutil import copyfile
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm as tq

# Libraries untuk pemrosesan data gambar
import cv2
from PIL import Image
import skimage
from skimage import io
from skimage.transform import resize
from skimage.transform import rotate, AffineTransform, warp
from skimage import img_as_ubyte
from skimage.exposure import adjust_gamma
from skimage.util import random_noise

# Libraries untuk pembangunan model
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import InputLayer, Conv2D, SeparableConv2D, MaxPooling2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau

In [270]:
!pip install kagglehub

## Data Preparation

### Data Loading

In [297]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (6).json


{'kaggle (6).json': b'{"username":"mutasar","key":"a9ebddb49d3274408fbb52fdb9dcdbee"}'}

In [300]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'


In [301]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [302]:
!kaggle datasets download -d moltean/fruits

Dataset URL: https://www.kaggle.com/datasets/moltean/fruits
License(s): CC-BY-SA-4.0
100% 3.46G/3.47G [01:27<00:00, 176MB/s]
100% 3.47G/3.47G [01:27<00:00, 42.5MB/s]


In [308]:
import os
print(os.listdir())


['.config', 'kaggle (4).json', 'fruits-360', 'fruit_images_data.csv', 'combined_dataset', 'kaggle (1).json', 'df_data.csv', 'kaggle (5).json', 'kaggle (2).json', 'kaggle.json', 'fruits.zip', 'file.csv', 'fruits-2000-sample', 'updated_fruit_data.csv', 'kaggle (6).json', 'kaggle (3).json', 'sample_data']


In [309]:
import zipfile

# Ekstrak file fruits-360.zip ke folder 'fruits-360'
with zipfile.ZipFile('fruits.zip', 'r') as zip_ref:
    zip_ref.extractall('fruits')


In [312]:
import os
print(os.listdir('fruits'))

['fruits-360_3-body-problem', 'fruits-360_100x100', 'fruits-360_multi', 'fruits-360_original-size', 'fruits-360_dataset_meta']


In [322]:
import os
print(os.listdir())  # Melihat folder di direktori kerja
print(os.listdir('fruits-360'))  # Melihat isi folder fruits-360


['.config', 'kaggle (4).json', 'fruits', 'fruits-360', 'fruit_images_data.csv', 'combined_dataset', 'kaggle (1).json', 'df_data.csv', 'kaggle (5).json', 'kaggle (2).json', 'kaggle.json', 'fruits.zip', 'file.csv', 'fruits-2000-sample', 'updated_fruit_data.csv', 'kaggle (6).json', 'kaggle (3).json', 'sample_data']
['fruits-360_3-body-problem', 'fruits-360_100x100', 'fruits-360_multi', 'fruits-360_original-size', 'fruits-360_dataset_meta']


In [314]:
import os
# Use the more likely correct path to the training data directory
training_data_path = 'fruits/fruits-360_100x100/fruits-360/Training'

# Check if the directory exists before listing
if os.path.exists(training_data_path):
    print(f"Listing contents of: {training_data_path}")
    print(os.listdir(training_data_path))
else:
    print(f"Error: Directory not found at {training_data_path}. Please verify the correct path based on your dataset extraction.")

Listing contents of: fruits/fruits-360_100x100/fruits-360/Training
['Cherry 4', 'Apple 11', 'Physalis with Husk 1', 'Zucchini 1', 'Tomato 10', 'Grapefruit Pink 1', 'Cucumber 3', 'Cherry Wax Red 2', 'Chestnut 1', 'Melon Piel de Sapo 1', 'Pear Williams 1', 'Potato Red Washed 1', 'Blueberry 1', 'Apple 6', 'Walnut 1', 'Onion White 1', 'Apple hit 1', 'Mulberry 1', 'Tomato 8', 'Apple 13', 'Zucchini dark 1', 'Avocado Green 1', 'Tomato 1', 'Cactus fruit green 1', 'Cabbage white 1', 'Cherry 2', 'Tomato 7', 'Banana 1', 'Carambula 1', 'Tomato not Ripen 1', 'Mango Red 1', 'Tomato Cherry Yellow 1', 'Potato Red 1', 'Cabbage red 1', 'Apple 19', 'Peach 1', 'Pepino 1', 'Nectarine 1', 'Hazelnut 1', 'Quince 2', 'Tomato Cherry Red 1', 'Cocos 1', 'Eggplant 1', 'Apple Golden 3', 'Tamarillo 1', 'Apple Pink Lady 1', 'Plum 2', 'Mangostan 1', 'Pear 1', 'Nut 3', 'Cucumber 4', 'Apple Red Yellow 1', 'Gooseberry 1', 'Redcurrant 1', 'Apricot 1', 'Kumquats 1', 'Tomato Cherry Maroon 1', 'Cherry Wax Red 1', 'Blackberri

In [317]:
import os
# Use the correct path to the training data directory
training_data_root = 'fruits/fruits-360_100x100/fruits-360/Training'

# Check if the training data root directory exists
if os.path.exists(training_data_root):
    classes = os.listdir(training_data_root)
    if classes:
        kelas = classes[0]  # Ambil nama kelas pertama
        print("Nama kelas pertama:", kelas)
        class_path = os.path.join(training_data_root, kelas)
        if os.path.isdir(class_path):
             print(f"Listing first 10 files in '{kelas}':")
             print(os.listdir(class_path)[:10])  # 10 file pertama
        else:
            print(f"Error: '{kelas}' is not a directory.")
    else:
        print(f"Error: No classes found in the training data root directory: {training_data_root}")

else:
    print(f"Error: Training data root directory not found at {training_data_root}. Please verify the correct path.")

Nama kelas pertama: Cherry 4
Listing first 10 files in 'Cherry 4':
['r0_258_100.jpg', 'r0_289_100.jpg', 'r2_304_100.jpg', 'r1_162_100.jpg', 'r1_98_100.jpg', 'r0_192_100.jpg', 'r1_201_100.jpg', 'r1_36_100.jpg', 'r2_173_100.jpg', 'r0_161_100.jpg']


### Data Preprocessing

In [323]:
# Define the correct paths for the training and testing directories
train_dir = 'fruits/fruits-360_100x100/fruits-360/Training'
test_dir = 'fruits/fruits-360_100x100/fruits-360/Test'

In [319]:
IMAGE_SIZE = 100  # Fruits-360: 100x100 px
BATCH_SIZE = 32
CHANNELS = 3

In [320]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)


In [324]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 103993 images belonging to 206 classes.
Found 34711 images belonging to 206 classes.


#### Split Dataset

In [326]:
from sklearn.model_selection import train_test_split

# Assuming you want to split the df_images DataFrame with this logic
# Ensure df_images is defined by running cell abac6d65 first
if 'df_images' in locals() and not df_images.empty:
    # Split df_images into train, validation, and test sets
    # Stratify by the 'class' column
    train_val_df, test_df = train_test_split(df_images, test_size=0.2, stratify=df_images['class'], random_state=42)
    train_df, val_df = train_test_split(train_val_df, test_size=0.25, stratify=train_val_df['class'], random_state=42) # 0.25 of 0.8 is 0.2

    # Results: 60% train, 20% val, 20% test from df_images
    print("Dataset split into train, validation, and test DataFrames (60/20/20 split on df_images).")
    print(f"Train DataFrame shape: {train_df.shape}")
    print(f"Validation DataFrame shape: {val_df.shape}")
    print(f"Test DataFrame shape: {test_df.shape}")
else:
    print("DataFrame 'df_images' not found or is empty. Cannot split dataset. Please ensure cell abac6d65 has been run.")

DataFrame 'df_images' not found or is empty. Cannot split dataset. Please ensure cell abac6d65 has been run.


In [285]:
img_size = (100, 100)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    work_test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 93656 images belonging to 206 classes.
Found 10337 images belonging to 206 classes.
Found 34711 images belonging to 206 classes.


In [287]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Input(shape=(224,224,3)),      # Layer Input eksplisit
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),                                  # Layer pooling kedua
    Flatten(),                                                        # Mengubah tensor 2D menjadi 1D
    Dense(64, activation='relu'),                                     # Fully connected layer
    Dense(10, activation='softmax')                                   # Output layer untuk 10 kelas
])



## Modelling

In [288]:
X = df['image_path']
y = df['split']  # Ganti sesuai nama kolom label di CSV Anda


In [289]:
from sklearn.model_selection import train_test_split

# Pembagian pertama: train dan sisa (val+test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Pembagian kedua: validation dan test dari sisa
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)


In [290]:
train_df = pd.DataFrame({'image_path': X_train, 'split': y_train})
val_df = pd.DataFrame({'image_path': X_val, 'split': y_val})
test_df = pd.DataFrame({'image_path': X_test, 'split': y_test})


In [291]:
from tensorflow.keras.layers import Input

# Ensure train_generator_df is defined before defining the model
if 'train_generator_df' in locals() and train_generator_df is not None:
    num_classes = len(train_generator_df.class_indices)
    print(f"Defining model with {num_classes} output classes.")

    model = Sequential([
        Input(shape=(224, 224, 3)), # Corrected Input layer shape
        Conv2D(32, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax') # Use num_classes from train_generator_df
    ])

    # Print the model summary to see the layers
    model.summary()

else:
    print("Error: train_generator_df not found. Please run cell 4be73d1f or b1234d03 to create data generators before defining the model.")
    model = None # Set model to None if generator is not found

Defining model with 206 output classes.


Model: "sequential_25"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_54 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_54 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_55 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_55 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_56 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_56 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_26 (Flatten)            │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 206)            │        52,942 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,297,614 (85.06 MB)

 Trainable params: 22,297,614 (85.06 MB)

 Non-trainable params: 0 (0.00 B)

In [295]:
# 1. Definisikan model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(224, 224, 3)),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# 2. Kompilasi model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 3. Training model
history = model.fit(
    train_generator_df,
    epochs=20,
    validation_data=val_generator_df
)


Epoch 1/20
169/169 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.0058 - loss: 10.3293

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 203), output.shape=(None, 206)

##Evaluasi dan Visualisasi

## Konversi Model

## Inference (Optional)

In [22]:
files.download('file.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>